# CW Sample Generator

In [ ]:
#!/usr/bin/env python
from scipy.io.wavfile import write as write_wav
import numpy as np
import IPython.display as ipd
from itertools import permutations
import random
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [ ]:
MORSE_CODE_DICT = { 'A':'.-', 'B':'-...', 
                    'C':'-.-.', 'D':'-..', 'E':'.', 
                    'F':'..-.', 'G':'--.', 'H':'....', 
                    'I':'..', 'J':'.---', 'K':'-.-', 
                    'L':'.-..', 'M':'--', 'N':'-.', 
                    'O':'---', 'P':'.--.', 'Q':'--.-', 
                    'R':'.-.', 'S':'...', 'T':'-', 
                    'U':'..-', 'V':'...-', 'W':'.--', 
                    'X':'-..-', 'Y':'-.--', 'Z':'--..', 
                    '1':'.----', '2':'..---', '3':'...--', 
                    '4':'....-', '5':'.....', '6':'-....', 
                    '7':'--...', '8':'---..', '9':'----.', 
                    '0':'-----', ',':'--..--', '.':'.-.-.-', 
                    '?':'..--..', '/':'-..-.', '-':'-....-', 
                    '(':'-.--.', ')':'-.--.-', ' ': ' '} 

In [ ]:
sample_rate = 44100
freq = 600 #Hz
wpm = 5.0

# how many seconds to jitter the samples
# this should allow for a better model by
# giving more than one "sample" for each character
JITTER_RANGE = 0.5

### Functions for generating tones and silence

The following functions will generate a tone or silence based on "time units"

These time units are all relative to eachother, and will ultimately change in length depending on the "wpm" variable

Silence is simply a list of zeroes.

Tones are a sin wave at freq Hz encoded into a list and normalized between -1 and 1.

In [ ]:
def generate_silence(time_units):
    return np.zeros(int(time_units * sample_rate / wpm))

def generate_tone(time_units):
    jitter = random.uniform(time_units*1.0-JITTER_RANGE,time_units*1.0+JITTER_RANGE)
    t = np.linspace(0.0, jitter / wpm, int(sample_rate*jitter/wpm))
    dit = np.sin(2.0 * np.pi * freq * t)
    return dit / max(abs(dit))

### Functions for generating dits, dahs, and various forms of silence

 * The length of a dot is 1 time unit.
 * A dash is 3 time units.
 * The space between symbols (dots and dashes) of the same letter is 1 time unit.
 * The space between letters is 3 time units.
 * The space between words is 7 time units.

In [ ]:
generate_word_sep = lambda: generate_silence(7)
generate_letter_sep = lambda: generate_silence(3)
generate_symbol_sep = lambda: generate_silence(1)
generate_dah = lambda: generate_tone(3) 
generate_dit = lambda: generate_tone(1)

**encode(s)** will encode a string of text into morse code audio.

The result is a list and is normalized between -1 and 1

In [ ]:
def encode(s):
    s = s.upper()
    result = generate_silence(random.uniform(5,15))
    for char in s:
        symbols = "'".join([i for i in MORSE_CODE_DICT[char]])
        for symbol in symbols:
            if symbol == '-':
                result = np.concatenate((result, generate_dah()))
            elif symbol == '.':
                result = np.concatenate((result, generate_dit()))
            elif symbol == "'":
                result = np.concatenate((result, generate_symbol_sep()))
            elif symbol == ' ':
                result = np.concatenate((result, generate_word_sep()))
        result = np.concatenate((result, generate_letter_sep()))
    return result

**SNR(cw, dB)** will add white noise at the given dB level (positive and negative numbers work)

In [ ]:
def SNR(cw, dB):
    SNR_linear = 10.0**(dB/10.0)
    power = cw.var()
    noise_power = power/SNR_linear
    noise = np.sqrt(noise_power)*np.random.normal(0,1,len(cw))
    return noise + cw

### Create WAV file
Here is where the actual wav gets created

In [ ]:
my_cq = SNR(encode("CQ CQ CQ DE KJ7PKQ"), -6)
write_wav("test.wav", sample_rate, my_cq.astype(np.float32))

In [ ]:
x, sr = librosa.load('test.wav')
%matplotlib inline
plt.figure(figsize=(14, 5))
_ = librosa.display.waveplot(x, sr=sr)

### Generate Spectrogram

In [ ]:
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
_ = librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

In [ ]:
ipd.Audio('test.wav')

### Code for Creating TensorFlow Inputs

In [ ]:
import cv2
from scipy.io import wavfile

from matplotlib.mlab import specgram
nfft = 256
overlap = nfft - 56  # overlap value for spectrogram

def get_specgram(signal, rate):
    arr2D, freqs, bins = specgram(
        signal,
        window=np.blackman(nfft),
        Fs=rate,
        NFFT=nfft,
        noverlap=overlap,
        pad_to=32 * nfft,
    )
    return arr2D, freqs, bins

def plot_image(arr2D, bins, freqs):
    fig, ax = plt.subplots(1,1)
    extent = (bins[0], bins[-1], freqs[-1], freqs[0])
    im = ax.imshow(
        arr2D,
        aspect="auto",
        extent=extent,
        interpolation="none",
        cmap="Greys",
        norm=None,
    )
    plt.gca().invert_yaxis()
    plt.show()

def normalize_image(img):
    # normalize
    (m, s) = cv2.meanStdDev(img)
    m = m[0][0]
    s = s[0][0]
    img = img - m
    img = img / s if s>0 else img
    return img


def create_image(filename):
    imgSize=(256, 32)
    dataAugmentation=False

    imgname = filename+".png"   

    # Load  image in grayscale if exists
    img = cv2.imread(imgname, 0) 

    # TODO: re-enable this IF statement
    #if img is None:
    rate, data = wavfile.read(filename)
    arr2D, freqs, bins = get_specgram(data, rate)

    # Get the image data array shape (Freq bins, Time Steps)
    shape = arr2D.shape

    # Find the CW spectrum peak - look across all time steps
    f = int(np.argmax(arr2D[:]) / shape[1])

    time_steps = (4.0/(len(data)/rate))*shape[1]

    # Create a 32x128 array centered to spectrum peak
    img = cv2.resize(arr2D[f - 16 : f + 16][:], imgSize)

    img = normalize_image(img)

    cv2.imwrite(imgname, img*256.)

    img = normalize_image(img)  
    # transpose for TF
    img = cv2.transpose(img)
    return img


### Create Training Image for Tensorlow

In [ ]:
create_image('test.wav')
from IPython.display import Image
Image(filename='test.wav.png') 

In [ ]:
def corpus(ngram):
    corpus = []
    for i in permutations(MORSE_CODE_DICT.keys(), ngram):
        corpus.append("".join(i))
    return corpus       